In [ ]:
#################################################################################################

In [97]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Selenium WebDriver 설정
driver = webdriver.Chrome()
url = "https://www.bmw.co.kr/ko/fastlane/dealer-locator/service-center.html"
driver.get(url)
time.sleep(5)  # 페이지 로드 대기

# iframe 전환
driver.switch_to.frame("iframepar__iframeadaptive")
time.sleep(2)  # iframe 내 요소 로드 대기

# 페이지 소스 가져오기
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")

# <ul class="content_area_choise_items"> 부분 추출
service_list = soup.find("ul", {"class": "content_area_choise_items"})

# 서비스 센터 정보 추출 및 데이터프레임 생성
parsed_data = []
if service_list:
    for li in service_list.find_all("li", class_="content_area_choise_item"):
        center_info = li.get_text(separator="\n", strip=True).splitlines()
        
        # 센터 정보 파싱
        center_name = center_info[0]  # 센터 이름
        
        phone = None
        new_address = None
        old_address = None
        business_hours = None
        electric_vehicle_service = None
        website = None

        for i, line in enumerate(center_info):
            if "전화번호" in line:
                phone = center_info[i + 1]
            elif "신주소" in line:
                new_address = center_info[i + 1]
            elif "구주소" in line:
                old_address = center_info[i + 1]
            elif "영업시간" in line:
                # 영업시간 추출
                hours_end_idx = next((j for j in range(i+1, len(center_info)) if "전기차" in center_info[j] or "홈페이지 바로 가기" in center_info[j]), len(center_info))
                business_hours = "\n".join(center_info[i+1:hours_end_idx]).strip()
            elif "전기차" in line:
                # 전기차 수리 관련 내용 추출
                ev_end_idx = next((j for j in range(i+1, len(center_info)) if "홈페이지 바로 가기" in center_info[j]), len(center_info))
                electric_vehicle_service = "\n".join(center_info[i:ev_end_idx]).strip()
            elif "홈페이지 바로 가기" in line:
                website = center_info[i + 1]
        
        # 추출한 정보를 딕셔너리에 저장
        parsed_data.append({
            "업체명": center_name,
            "전화번호": phone,
            "도로명주소": new_address,
            "주소": old_address,
            "영업시간": business_hours,
            "전기차 서비스": electric_vehicle_service,
            "링크": website
        })

    # 데이터프레임 생성
    df_centers = pd.DataFrame(parsed_data)
else:
    print("서비스 센터 목록을 찾을 수 없습니다. 페이지 구조가 변경되었을 수 있습니다.")

In [99]:
df_centers

,업체명,전화번호,도로명주소,주소,영업시간,전기차 서비스,링크
0,강동 패스트레인 서비스 센터 (도이치 모터스),02-488-8766,서울 강동구 양재대로 1571 홈플러스강동점 지하 4층,서울 강동구 천호동 42 홈플러스강동점 지하 4층,-평일: 10:00 ~ 19:00\n-토요일: 10:00 ~ 17:00,None,https://www.deutschmotors.com/
1,강릉 서비스 센터 (코오롱 모터스),033) 800-7301,강원특별자치도 강릉시 율곡로 3064,강원 강릉시 교동 613-1,평일: 09:00~18:00\n토요일: 휴무\n일요일: 휴무,None,https://kolonmotors.com
2,강북 서비스 센터 (한독 모터스),02)3444-7301,서울특별시 강북구 도봉로 101,서울특별시 강북구 미아동 374-1,"-평일: 09:00~18:00\n-토요일: 09:00~16:00\n-일요일,공휴일: 휴무",None,http://www.bmwhandok.co.kr/
3,강서 서비스 센터 (바바리안 모터스),02)2661-7401,서울특별시 강서구 공항대로 59,서울특별시 강서구 공항동 14-81,-평일: 08:30 ~ 17:30\n-토요일: 09:00 ~ 13:00\n-일요일:휴무,전기차에 대한 진단 및 고전압 시스템을 제외한 일반 정비 가능,http://www.bavarian.co.kr/
4,고양 IKEA 패스트레인 서비스 센터 (바바리안 모터스),031-994-7302,"경기도 고양시 덕양구 권율대로 420 (도내동), 이케아 고양점 P2 주차장","경기도 고양시 덕양구 도내동 1003, 이케아 고양점 P2 주차장","-평일: 10:00 ~ 19:00\n-토요일: 10:00 ~ 14:00\n-일요일,...",None,http://www.bavarian.co.kr/
...,...,...,...,...,...,...,...
73,청주 서비스 센터 (삼천리 모터스),043)236-7501,충청북도 청주시 흥덕구 가로수로 754번길 15,충청북도 청주시 흥덕구 석소동 13-18,-평일: 09:00 ~ 18:00\n-토요일: 09:00 ~ 13:00\n-일요일:휴무,전기차 수리가능 등급 : HV\n- 고전압시스템을 포함한 모든 수리,http://www.samchullymotors.co.kr
74,춘천 서비스 센터 (코오롱 모터스),033)830-7301,강원특별자치도 춘천시 동면 춘천순환로 852,강원도 춘천시 동면 장학리 502-2,-평일: 09:00 ~ 18:00\n-토요일: 휴무\n-일요일: 휴무,None,https://www.kolonmotors.com/
75,평택 서비스 센터 (내쇼날 모터스),031)656-7501,경기도 안성시 원곡면 청원로 1797,경기도 안성시 원곡면 내가천리 158-8,-평일: 08:30~17:30\n-토요일: 08:30~15:00\n-일요일: 휴무,None,http://www.nationalmotors.co.kr/
76,포항 서비스 센터 (동성 모터스),054)272-7306,경상북도 포항시 북구 새천년대로 569,경상북도 포항시 북구 용흥동 624-5,-평일: 10:00 ~ 19:00\n-토요일: 10:00 ~ 15:00\n-일요일: 휴무,None,http://www.bmwdongsung.co.kr/


In [101]:
df = df_centers

In [103]:
df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시', '', regex=True)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

In [107]:
df.to_excel('BMW_최종.xlsx')